## Effect of fc'-embodied carbon relationship on structural components' embodied carbon

Activate and check packages

In [1]:
using Pkg
Pkg.activate("..");
Pkg.status()

  Activating project at `~/dev/ConcreteExplorer`


Status `~/dev/ConcreteExplorer/Project.toml`
  [acfe5cae] AsapSections v0.1.0 `https://github.com/keithjlee/AsapSections#main`
  [336ed68f] CSV v0.10.11
  [a93c6f00] DataFrames v1.6.1
  [e9467ef8] GLMakie v0.9.1
  [682c06a0] JSON v0.21.4
  [ee78f7c6] Makie v0.20.1
  [90137ffa] StaticArrays v1.7.0


In [2]:
using AsapSections

Define constants
based on ASCE with some assumptions on super imposed deadload

In [111]:
# function get_beam_carbon(fc′::Float64, ec::Float64, section::SolidSection;

# wLL =  1.92 #kN/m2
# wSDL = 1.5 #kN/m2, concrete 1" finishing, 32 psf.
@show wLL = 1.92e-3 #N/mm2
@show wSDL = 1.5e-3 #N/mm2

@show ρ_concrete = 24000.0e-9 #N/mm3

#Structural spans
span = 5000.0 #mm
bay  = 5000.0 #mm

wLL = 0.00192 = 0.00192
wSDL = 0.0015 = 0.0015
ρ_concrete = 2.4e-5 = 2.4e-5


5000.0

Dummy variables that will be turned into function's inputs

In [125]:
# Dummy
fc′ = 28.0 #MPa -> N/mm2
fy = 420.0 #MPa -> N/mm2
Es = 210_000 #MPa
ec = 0.1 #kgCO2e/kg concrete

bd_ratio = 0.5
d = 300.0 #mm
b = bd_ratio*d #mm

p1 = [0.0 , 0.0]
p2 = [b, 0.0]
p3 = [b, -d]
p4 = [0.0, -d]

points = [p1, p2, p3, p4]
section = CompoundSection([SolidSection(points)])
section_area = section.area

45000.0

In [126]:
fieldnames(CompoundSection)
d_cal = section.ymax -section.ymin
@assert d_cal == d

Calculating moment and shear demand

In [127]:
# Demands
@show w_load = (1.2*wSDL + 1.6*wLL)*bay
@show w_beamDL = 1.2*ρ_concrete*section_area
@show w = w_load + w_beamDL
Mdemand = w * span^2/8
Vdemand = w * span/2
println("Moment Demand: ", Mdemand)
println("Shear Demand: ", Vdemand)

w_load = (1.2wSDL + 1.6wLL) * bay = 24.36
w_beamDL = 1.2 * ρ_concrete * section_area = 1.296
w = w_load + w_beamDL = 25.656
Moment Demand: 8.0175e7
Shear Demand: 64140.0


Use the demands and fc' to get section's reinforcement ratio<br>
ρ_min is defined by the requires demands<br>
ρ is capped (ρ_max) by the ductility of the section (ϵₛ> 0.005)



In [128]:
@show a = Mdemand/(0.85*fc′*b)

as = Mdemand/(fy*(d-a/2))
ϵs_calc = 0.85*fc′*a*b/Es

a = Mdemand / (0.85 * fc′ * b) = 22457.983193277312


381.7857142857143

In [ ]:
#use Mdemand and Vdemand to find the right reinforcement steel ratio (ρ_s)
#area of steel is ρ_s*section_area(make sure you substract area of steel out too)




#ρ_min is defined by the requires demands
ρ_min = 

#ρ is capped (ρ_max) by the ductility of the section (ϵₛ> 0.005)
ρ_max = 
ρ_selected = (ρ_min+ρ_max)/2

reinforcement_area = section_area*ρ_selected

Now we have a concrete section, we can define a ReinforcedConcrete section struct, along with calculations for demands.

In [ ]:
struct